In [1]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import pymesh
import meshio
import math as m

from mesh3d import Mesh3D

%matplotlib inline
plt.rcParams["figure.figsize"] = 12.8, 9.6

In [27]:
CELL_SIZE=5e-1

# Test mesh loading, saving & resizing

In [3]:
input_mesh = pymesh.load_mesh("meshes/sausage_shape.obj")

mesh = Mesh3D(input_mesh, CELL_SIZE)

pymesh.save_mesh("meshes/regularised_cell.ply", mesh.input_mesh)
mesh.write_input_mesh("meshes/regularised_cell.vtk")

mesh.write_mesh("meshes/regularised_cell_tetra.vtk")

# Test boundary node marking

In [4]:
tetmesh = mesh.tetmesh
boundary_mesh = pymesh.compute_outer_hull(mesh.tetmesh)

is_boundary = np.zeros(tetmesh.num_vertices)
source_faces = boundary_mesh.get_attribute("face_sources").astype(int)
for i_face in source_faces:
    face = tetmesh.faces[i_face]
    is_boundary[face] = True

mesh.set_attribute("is_boundary", is_boundary)

mesh.write_mesh("meshes/regularised_cell_tetra_with_boundary.vtk")

In [5]:
tetmesh = mesh.tetmesh
boundary_mesh = pymesh.compute_outer_hull(mesh.tetmesh)

is_boundary = np.zeros(tetmesh.num_vertices)
source_faces = boundary_mesh.get_attribute("face_sources").astype(int)
boundary_vertices = np.unique(np.array(tetmesh.faces[source_faces]).flatten())

is_boundary[boundary_vertices] = True

mesh.set_attribute("is_boundary", is_boundary)

mesh.write_mesh("meshes/regularised_cell_tetra_with_boundary.vtk")

# Test node data output

In [6]:
vertex_data = tetmesh.vertices[:, 2]
mesh.set_attribute("data", vertex_data)

mesh.write_mesh("meshes/regularised_cell_tetra_with_data.vtk")

# Test steady-state heat equation

## Dirichlet Condition 1

In [28]:
input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=2)
test_mesh = Mesh3D(input_mesh, CELL_SIZE)
test_mesh.write_mesh("meshes/sphere_mesh.vtk")

In [29]:
print("Number of vertices:", test_mesh.n_vertices)

Number of vertices: 163


In [30]:
def cart2sph(x,y,z):
    XsqPlusYsq = x**2 + y**2
    r = m.sqrt(XsqPlusYsq + z**2)               # r
#     elev = m.atan2(z,m.sqrt(XsqPlusYsq))     # theta
    elev = m.acos(z/r)     # theta
    az = m.atan2(y,x)                           # phi
    return r, elev, az

In [31]:
def calculate_dirichlet_values(vertex):
    r, theta, phi = cart2sph(vertex[0], vertex[1], vertex[2])
    return m.cos(theta)**2

stiffness = test_mesh.assemble_stiffness()
stiffness = test_mesh.assembler.assemble("laplacian")
print(stiffness.shape)
stiffness = stiffness.todense()
print(stiffness.shape)

# Remove dirichlet boundary points
stiffness[test_mesh.boundary_vertices,:] = 0.0
# And ensure matrix isn't singular
for vertex in test_mesh.boundary_vertices:
    stiffness[vertex, vertex] = 1.0
    
rhs_vector = np.zeros(test_mesh.n_vertices)

dirichlet_rhs = np.zeros(test_mesh.n_vertices)
dirichlet_rhs[test_mesh.boundary_vertices] = test_mesh.calculate_boundary_values(calculate_dirichlet_values)

rhs_vector += dirichlet_rhs

# Solve
soln = np.linalg.solve(stiffness, rhs_vector)

(163, 163)
(163, 163)


In [32]:
def calc_analytical_solution(vertex):
    r, theta, phi = cart2sph(vertex[0], vertex[1], vertex[2])
    c = [1, 0, 2*(r)**2]
    return 1/3*np.polynomial.legendre.legval(m.cos(theta), c)

analytical_soln = np.array([calc_analytical_solution(v) for v in test_mesh.vertices()])

# Calculate error
error = np.abs(soln - analytical_soln)

print("Error is", np.linalg.norm(error))

test_mesh.set_attribute("u", soln)
test_mesh.set_attribute("error", error)
test_mesh.set_attribute("analytical_u", analytical_soln)

test_mesh.write_mesh("meshes/laplace_equation_dirichlet_result.vtk")

Error is 1.132892344728772e-15


## Dirichlet condition 2

In [33]:
input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=2)
test_mesh = Mesh3D(input_mesh, CELL_SIZE)
# test_mesh.write_mesh("meshes/sphere_mesh.vtk")

In [34]:
def cart2sph(x,y,z):
    XsqPlusYsq = x**2 + y**2
    r = m.sqrt(XsqPlusYsq + z**2)               # r
#     elev = m.atan2(z,m.sqrt(XsqPlusYsq))     # theta
    elev = m.acos(z/r)     # theta
    az = m.atan2(y,x)                           # phi
    return r, elev, az

In [35]:
def calculate_dirichlet_values(vertex):
    r, theta, phi = cart2sph(vertex[0], vertex[1], vertex[2])
    return m.cos(theta)

# stiffness = test_mesh.assemble_stiffness()
stiffness = test_mesh.assembler.assemble("laplacian")
print(stiffness.shape)
stiffness = stiffness.todense()
print(stiffness.shape)

# Remove dirichlet boundary points
stiffness[test_mesh.boundary_vertices,:] = 0.0
# And ensure matrix isn't singular
for vertex in test_mesh.boundary_vertices:
    stiffness[vertex, vertex] = 1.0
    
rhs_vector = np.zeros(test_mesh.n_vertices)

dirichlet_rhs = np.zeros(test_mesh.n_vertices)
dirichlet_rhs[test_mesh.boundary_vertices] = test_mesh.calculate_boundary_values(calculate_dirichlet_values)

rhs_vector += dirichlet_rhs

# Solve
soln = np.linalg.solve(stiffness, rhs_vector)

(163, 163)
(163, 163)


In [36]:
def calc_analytical_solution(vertex):
    return vertex[2]

analytical_soln = np.array([calc_analytical_solution(v) for v in test_mesh.vertices()])

# Calculate error
error = np.abs(soln - analytical_soln)

print("Error is", np.linalg.norm(error))

test_mesh.set_attribute("u", soln)
test_mesh.set_attribute("error", error)
test_mesh.set_attribute("analytical_u", analytical_soln)

test_mesh.write_mesh("meshes/laplace_equation_dirichlet_2_result.vtk")

Error is 1.2506214487394732e-15


## von Neumann conditions

In [48]:
input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=2)
test_mesh = Mesh3D(input_mesh, CELL_SIZE)
test_mesh.write_mesh("meshes/sphere_mesh.vtk")

In [49]:
def cart2sph(x,y,z):
    XsqPlusYsq = x**2 + y**2
    r = m.sqrt(XsqPlusYsq + z**2)               # r
#     elev = m.atan2(z,m.sqrt(XsqPlusYsq))     # theta
    elev = m.acos(z/r)     # theta
    az = m.atan2(y,x)                           # phi
    return r, elev, az

In [50]:
def calculate_von_neumann_values(vertex):
    r, theta, phi = cart2sph(vertex[0], vertex[1], vertex[2])
    return m.cos(theta)
#     return 0.0

stiffness = test_mesh.assembler.assemble("laplacian")
stiffness = stiffness.todense()
  
rhs_vector = np.zeros(test_mesh.n_vertices)

# Incorporate von Neumann condition
boundary_values = test_mesh.calculate_boundary_values(calculate_von_neumann_values)
rhs_vector += test_mesh.assemble_von_neumann_boundary(boundary_values)

# Incorporate Dirichlet condition
fixed_boundary_vertex = test_mesh.boundary_vertices[0]

# Remove dirichlet boundary points
stiffness[fixed_boundary_vertex,:] = 0.0
# And ensure matrix isn't singular
for vertex in [fixed_boundary_vertex]:
    stiffness[vertex, vertex] = 1.0

rhs_vector[fixed_boundary_vertex] = test_mesh.get_pos(fixed_boundary_vertex)[2]

# Solve
soln = np.linalg.solve(stiffness, rhs_vector)

In [51]:
def calc_analytical_solution(vertex):
    return vertex[2]

analytical_soln = np.array([calc_analytical_solution(v) for v in test_mesh.vertices()])

# Calculate error
error = np.abs(soln - analytical_soln)

print("Error is", np.linalg.norm(error))

test_mesh.set_attribute("u", soln)
test_mesh.set_attribute("error", error)
test_mesh.set_attribute("analytical_u", analytical_soln)

test_mesh.write_mesh("meshes/laplace_equation_von_neumann_result.vtk")

Error is 1.9459738619672986
